## Export GitHub Commits to CSV

### Input

In [1]:
year = 2023
org = 'examind-ai'

### Setup

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

### HTTP Request

In [3]:
import os
import requests

headers = {
    'Authorization': f'token {os.getenv("GITHUB_ACCESS_TOKEN")}',
    'Accept': 'application/vnd.github.v3+json',
}

def fetch_page_items(url):
    response = requests.get(url, headers=headers)
    items = response.json()
    next_page_url = None

    # Check if there is a 'Link' header and parse for next page URL
    if 'link' in response.headers:
        links = response.headers['link'].split(', ')
        next_link = [link for link in links if 'rel="next"' in link]
        if next_link:
            next_page_url = next_link[0].split(';')[0].strip('<>')

    return items, next_page_url

def fetch_all_items(url):
    all_items = []
    next_url = url
    while next_url:
        items, next_url = fetch_page_items(next_url)
        all_items.extend(items)

    return all_items

### Export to CSV

In [4]:
from datetime import datetime
import csv

filename = f'{year}_{org}_commits.csv'
filepath = os.path.join(os.path.expanduser('~'), 'Downloads', filename)


if os.path.exists(filepath):
    raise FileExistsError(f"The file '{filepath}' already exists. Please remove it first.")

since = datetime(year, 1, 1).isoformat() + 'Z'
until = datetime(year + 1, 1, 1).isoformat() + 'Z'

repos_url = f'https://api.github.com/orgs/{org}/repos'

# Define the CSV header
fields = ['Repository', 'SHA', 'Author', 'Date', 'Message']

repos = fetch_all_items(repos_url)

with open(filepath, 'w', newline='', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields)  # Write the CSV header

    for repo in repos:
        repo_name = repo['name']
        print(f"Fetching commits for '{repo_name}'...")
    
        commits_url = f'https://api.github.com/repos/{org}/{repo_name}/commits?since={since}&until={until}'
        commits = fetch_all_items(commits_url)

        for commit in commits:
            sha = commit['sha']
            author = commit['commit']['author']['name']
            date = commit['commit']['author']['date']
            message = commit['commit']['message']
            csvwriter.writerow([repo_name, sha, author, date, message])

print(f"Commits saved to '{filepath}'.")

Fetching commits for 'examind-web'...
Fetching commits for 'examind-moderator'...
Fetching commits for 'examind-firebase'...
Fetching commits for 'PrairieLearn'...
Fetching commits for 'pl-demo-course'...
Fetching commits for 'examind-question-builder'...
Fetching commits for 'examind-demo-widget-tester'...
Fetching commits for 'blackboard-export-viewer'...
Fetching commits for 'froala-source'...
Fetching commits for 'examind-froala'...
Fetching commits for 'qti-export-viewer'...
Fetching commits for 'examind-blockly'...
Fetching commits for 'json-2-csv'...
Fetching commits for 'ltijs-demo-server'...
Fetching commits for 'examind-io'...
Fetching commits for 'mui-submodule-to-top-level-imports'...
Fetching commits for 'examind-access-code'...
Fetching commits for 'examind-access-code-cf'...
Fetching commits for 'examind-lti-proxy'...
Fetching commits for 'ltijs-firestore'...
Fetching commits for 'ltijs-firestore-scheduler'...
Fetching commits for 'react-floater'...
Fetching commits for 